In [1]:
link = "https://www.reuters.com/resources/archive/us/{}{}{}.html"
ag = "Reuters"
#reuters_db = '/home/ostapkharysh/Documents/bt_data/DB/Reuters'

In [2]:
from bs4 import BeautifulSoup as BS
import requests
from requests.exceptions import HTTPError
import datetime
#from db_manager import *
import sqlite3
from tqdm import tqdm
from multiprocessing import Pool
from manager import *
from threading import Lock
mutex = Lock()

years = ['2013']#, '2014', '2015', '2016', '2017']
months = ['01',]# '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = ['01', ]#'02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19'
       #'20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
    
exec(open("db_management/DB.py").read()) 

In [3]:
add_agency(ag)

[]


'The new table Reuters is created.'

In [4]:
def get_text(link):
    try:
        r = requests.get(link)
        #print(r.raise_for_status())
        soup = BS(r.content, 'html.parser')
        head = soup.html.title.string.strip().split(" | ")[0]
        text = '\n'.join([e.get_text() for e in soup.find_all('p')])
        #print(text)
        #print(head)
        return head, text
    
    except HTTPError:
        print("No article found by this link!", link)
            

In [5]:
def get_info(link_time): 
    try:
    #for link_time in links_times:
        title,text = get_text(link_time[0])
        time = link_time[1]
        date = datetime.datetime.strptime("{} {} {} {}".format(month, day, year, time), '%b %d %Y %I:%M%p')

        #print(str(date)+'\n'+title+'\n'+text+'\n'+link_time[0])
        #print(str(date)+'\n'+title+'\n'+link_time[0])
        mutex.acquire()
        try:
            add_news(str(date), title, text, link_time[0], ag)
        finally:
            mutex.release()
    except HTTPError:
        print("No page found by this link!",r.status)

In [7]:

for y in years:
    for m in months:
        for d in days:
            try:
                datetime.datetime(year=int(y),month=int(m),day=int(d))
                r = requests.get(link.format(y, m, d))
                r.raise_for_status() # check if page found
                
                soup = BS(r.content, 'html.parser')
                span_list = soup.find_all('div' , attrs={'class':"headlineMed"})
                header = str(soup.find_all('title')).split()
                day, month, year = header[6], header[7], header[8]
                
                links_times = [[str(el).split('>')[1].split("=")[1].replace('"', ''), str(el).split()[-2]] for el in span_list 
                               if not str(el).split('>')[1].split("=")[1].replace('"', '').startswith('http://www.reuters.com/news/video')]
            except HTTPError:
                print("No page found by this link!",r.status)
                continue
            except ValueError:
                print("Day doesn't exist")
                continue
            #numbers = [5, 10, 20, 1, 2, 3, 4, 7,7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
            pool = Pool(processes=10)
            pool.map(get_info, links_times)
                    
print("DONE")

DONE
